In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

We are tasked in developing an algorithm in order to detect certain lowercase cursive letters. The program should be able to detect which letter is in an imported image and output the accuracy of that letter being the predicted letter. 

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [ ]:
import zipfile
import os

# Define base and data paths
base_path = "/home/jupyter-1016468/Git-Repo-Tour/Project_5"
data_dir = os.path.join(base_path, "Data")

# Define ZIP file paths
cursive_zip = os.path.join(data_dir, "Cursive.zip")
kaggle_zip = os.path.join(data_dir, "archive.zip")

# Define extraction directories
cursive_extract_path = os.path.join(data_dir, "Cursive")
kaggle_extract_path = os.path.join(data_dir, "Signatures")

# Make sure folders exist
os.makedirs(cursive_extract_path, exist_ok=True)
os.makedirs(kaggle_extract_path, exist_ok=True)

# --- Unzip the Cursive dataset ---
try:
    with zipfile.ZipFile(cursive_zip, 'r') as zip_ref:
        zip_ref.extractall(cursive_extract_path)
    print(f"Successfully extracted Cursive.zip to: {cursive_extract_path}")
except FileNotFoundError:
    print("Error: Could not find Cursive.zip in the Data folder.")
except zipfile.BadZipFile:
    print("Error: Cursive.zip appears to be corrupted or invalid.")

# --- Unzip the Kaggle signature dataset ---
try:
    with zipfile.ZipFile(kaggle_zip, 'r') as zip_ref:
        zip_ref.extractall(kaggle_extract_path)
    print(f"Successfully extracted archive.zip to: {kaggle_extract_path}")
except FileNotFoundError:
    print("Error: Could not find archive.zip in the Data folder.")
except zipfile.BadZipFile:
    print("Error: archive.zip appears to be corrupted or invalid.")


# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [ ]:
import glob
import cv2
import random
import matplotlib.pyplot as plt

jpg_files = glob.glob(f"{extract_path}/**/*.jpg", recursive=True)
heic_files = glob.glob(f"{extract_path}/**/*.heic", recursive=True)

print("JPG count:", len(jpg_files))
print("HEIC count:", len(heic_files))


sample = random.choice(jpg_files)
img = cv2.imread(sample)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title(f"Sample image: {sample.split('/')[-1]}")
plt.axis('off')
plt.show()



# 4.Prepare the Data


Apply any data transformations and explain what and why


In [ ]:
import os
import glob
import cv2
from PIL import Image
import numpy as np

# Paths
base_path = "Data/Cursive"  # relative to Project_5 notebook
subject_folders = [f"S{i}" for i in range(1, 36)]

# Storage
viable_images = []
corrupted_images = []

# Image size
IMG_SIZE = (64, 64)

# Loop through all subject folders
for folder in subject_folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        continue
    for img_file in glob.glob(f"{folder_path}/*.*"):
        try:
            # Convert HEIC to JPG if needed
            if img_file.lower().endswith(".HEIC"):
                img_jpg = img_file.replace(".HEIC", ".jpg")
                with Image.open(img_file) as im:
                    im.convert("RGB").save(img_jpg, "JPEG")
                img_file = img_jpg
            
            # Try reading with OpenCV
            img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
            if img is None:
                corrupted_images.append(img_file)
                continue
            
            # Resize and normalize
            img = cv2.resize(img, IMG_SIZE)
            img = img / 255.0
            
            # Store image + label (subject folder)
            viable_images.append((img, folder))
        except Exception as e:
            corrupted_images.append(img_file)

print(f"Total images found: {len(viable_images) + len(corrupted_images)}")
print(f"Viable images: {len(viable_images)}")
print(f"Corrupted/unreadable images: {len(corrupted_images)}")


In [ ]:
import os
import glob
import cv2
from PIL import Image

base_path = "Data/Cursive"
subject_folders = [f"S{i}" for i in range(1, 36)]

viable_images = []
corrupted_images = []
converted_count = 0

for folder in subject_folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        continue
    
    for img_file in glob.glob(f"{folder_path}/*.*"):
        # Convert to JPG if not already
        if not img_file.lower().endswith(".jpg"):
            try:
                with Image.open(img_file) as im:
                    im = im.convert("RGB")  # ensure it's RGB
                    new_path = os.path.splitext(img_file)[0] + ".jpg"
                    im.save(new_path, "JPEG")
                    converted_count += 1
                    os.remove(img_file)  # remove original
                    img_file = new_path  # update path to new jpg
            except Exception as e:
                print(f"Failed to convert {img_file}: {e}")
                corrupted_images.append(img_file)
                continue
        
        # Check if OpenCV can read it
        try:
            img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
            if img is None:
                corrupted_images.append(img_file)
            else:
                viable_images.append(img_file)
        except Exception as e:
            corrupted_images.append(img_file)

print(f"Conversion complete! Total files converted to JPG: {converted_count}")
print(f"Total viable images: {len(viable_images)}")
print(f"Total corrupted/unreadable images: {len(corrupted_images)}")


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 
